In [1]:
import pandas as pd
import json

In [2]:
import topic_modeler.gizmodo_modeler
import importlib
import topic_modeler.modeling
importlib.reload(topic_modeler.modeling)
importlib.reload(topic_modeler.gizmodo_modeler)
from topic_modeler.gizmodo_modeler import GizmodoModeler

Using TensorFlow backend.


In [3]:
gizmodeler  = GizmodoModeler()

In [4]:
gizmodeler.drop_gizmodo_site('io9.gizmodo.com')
gizmodeler.articles.cluster.value_counts()

theonion.com           2997
theroot.com            2992
kotaku.com             2992
deadspin.com           2954
lifehacker.com         2924
jezebel.com            2920
thetakeout.com         2900
gizmodo.com            2815
jalopnik.com           2798
earther.gizmodo.com    2525
Name: cluster, dtype: int64

In [5]:
model_cfg = gizmodeler.cfg['TOPIC_MODELING']
random_state = model_cfg.getint('RANDOM_STATE')

In [6]:
gizmodeler.train_test_split_articles(random_state=random_state)

{'X_train':                                                body_text
 20495  In July 1959, a young and disconcertingly hand...
 26452  It’s time. Time for us to get behind the wheel...
 3224   With E3 coming up in less than a month, it’s t...
 21392  Earlier this summer, Christian “humor” site Th...
 6230   Every video game has its peaks and valleys. A ...
 ...                                                  ...
 17209  In August, Minneapolis banned the construction...
 5437   Pass that dutch—it’s a celebration, bitches! M...
 5806   Authorities ushered a man out of a Vermont cou...
 410    Gita Jackson, Kotaku staff writer and likely S...
 4490   Three couples in Virginia are invoking the nam...
 
 [23053 rows x 1 columns],
 'X_test':                                                body_text
 28793  Israel was allegedly behind the Stingray spy d...
 22606  Avisaíl García, the disorienting ceiling at th...
 23013  On Monday, Politico ran a lengthy piece levera...
 6982   The U.S. gover

In [7]:
gizmodeler.vectorize_articles(**json.loads(model_cfg['ARTICLE_VECTORIZER_SETTINGS']))

Vectorizing articles...


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['concourse', 'muse', 'onion', 'root', 'slot', 'takeout'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


""
0
1
2
3
4
...
29672
29673
29674
29675


In [8]:
gizmodeler.splitted_vectorized_articles['X_train']

<23053x100000 sparse matrix of type '<class 'numpy.float64'>'
	with 4620468 stored elements in Compressed Sparse Row format>

In [9]:
gizmodeler.splitted_articles['y_train'].value_counts()

theonion.com           2397
theroot.com            2394
kotaku.com             2394
deadspin.com           2363
lifehacker.com         2339
jezebel.com            2336
thetakeout.com         2320
gizmodo.com            2252
jalopnik.com           2238
earther.gizmodo.com    2020
Name: cluster, dtype: int64

In [10]:
model_cfg = gizmodeler.cfg['TOPIC_MODELING']
model_cfg['MODEL_SETTINGS'] 

'{ "solver" : "saga", "multi_class" : "multinomial", "penalty" : "l1", "max_iter" : 1000, "n_jobs" : -1, "resampled" : false }'

In [11]:
json.loads(model_cfg['MODEL_SETTINGS'])

{'solver': 'saga',
 'multi_class': 'multinomial',
 'penalty': 'l1',
 'max_iter': 1000,
 'n_jobs': -1,
 'resampled': False}

In [12]:
gizmodeler.train_article_classifier(classifier=model_cfg['MODEL'], 
                                 **json.loads(model_cfg['MODEL_SETTINGS']),
                                 random_state=random_state
                                )

Training LogisticRegression classifier with params {'solver': 'saga', 'multi_class': 'multinomial', 'penalty': 'l1', 'random_state': 12, 'max_iter': 1000, 'C': 0.1, 'n_jobs': -1}...


In [13]:
gizmodeler.display_classifier_accuracy()

Test accuracy: 0.6471
% non-zero coefficients per class:
 [0.024 0.027 0.019 0.015 0.015 0.013 0.02  0.008 0.011 0.02 ]
                     precision    recall  f1-score   support

       deadspin.com       0.77      0.64      0.70       591
earther.gizmodo.com       0.79      0.78      0.78       505
        gizmodo.com       0.59      0.48      0.53       563
       jalopnik.com       0.87      0.75      0.81       560
        jezebel.com       0.49      0.48      0.49       584
         kotaku.com       0.75      0.72      0.73       598
     lifehacker.com       0.54      0.62      0.57       585
       theonion.com       0.46      0.71      0.56       600
        theroot.com       0.63      0.63      0.63       598
     thetakeout.com       0.78      0.68      0.73       580

           accuracy                           0.65      5764
          macro avg       0.67      0.65      0.65      5764
       weighted avg       0.67      0.65      0.65      5764

